In [2]:
import pandas as pd
import uuid
import random
import json

### Fix on Google Drive

In [ ]:
# df = pd.read_pickle('../data/teaching.pkl')
# sermons = df[df.category=='sermon'].rename(columns={'title':'title_raw'})
# def get_title(s):
#     try:
#         return s[:s.rfind('(')].strip()
#     except:
#         return ''

# def get_passage(s):
#     try:
#         return s[s.rfind('('):].strip().replace('(','').replace(')','')
#     except:
#         return ''
                
# sermons['title'] = sermons.title_raw.apply(lambda x: get_title(x))
# sermons['passage'] = sermons.title_raw.apply(lambda x: get_passage(x))
# sermons = sermons.sort_values('date_str').reset_index(drop=True)
# sermons.to_excel('../data/temp_sermons_for_google_drive.xlsx')

### Prep for Sanity schema

In [7]:
def new_audio_link(audioLink):
    try:
        return audioLink.split('.org/')[-1].replace('/','___')
    except:
        return ""

In [ ]:
sermons_edited = pd.read_excel('../data/sermons.xlsx')


In [ ]:
sermons_edited = sermons_edited.drop(columns=['Unnamed: 0',])
sermons_edited['mp3File'] = sermons_edited.audioLink.apply(lambda x: new_audio_link(x))
sermons_edited = sermons_edited.drop(columns=['date'])
sermons_edited = sermons_edited.rename(columns={'date_str':'date', 'audioLink':'oldAudioLink'})
sermons_edited = sermons_edited.reset_index().rename(columns={'index':'_id', 'series':'sermonSeries'})
sermons_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(sermons_edited.shape[0])]
sermons_edited['_type'] = 'sermons'
sermons_edited = sermons_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sermonSeries', 'mp3File', 'oldAudioLink']]
sermons_edited = sermons_edited.fillna("")

In [ ]:
sermons_edited = sermons_edited[sermons_edited.date <= "2022-10-31"]

sermons_edited['book'] = sermons_edited.book.apply(lambda x: 'Hebrews' if x == 'Hebrew' else x)

sermons_edited.drop(columns=['mp3File']).to_json("../cms/sermons.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)


In [ ]:
sermons_edited.head(2)

In [ ]:
mp3List = sermons_edited[['_id','mp3File']].rename(columns={'_id':'assetId', 'mp3File':'mp3FileName'}).to_dict('records')
with open('../cms/mp3List.js', 'w') as f:
    f.write(f"export const mp3s = {json.dumps(mp3List)}")

### Sunday School for Sanity

In [ ]:
# df = pd.read_pickle('../data/teaching_clean.pkl')
# ss = df[df.category=='sunday school'].rename(columns={'title':'title_raw'})
# def get_title(s):
#     try:
#         return s[:s.rfind('(')].strip()
#     except:
#         return ''

# def get_passage(s):
#     try:
#         return s[s.rfind('('):].strip().replace('(','').replace(')','')
#     except:
#         return ''
                
# ss['title'] = ss.title_raw.apply(lambda x: get_title(x))
# ss['passage'] = ss.title_raw.apply(lambda x: get_passage(x))
# ss = ss.sort_values('date_str').reset_index(drop=True)
# ss.to_excel('../data/temp_ss_for_google_drive.xlsx')

In [8]:
ss_edited = pd.read_excel('../data/temp_ss_for_google_drive_fixed.xlsx')

In [9]:
ss_edited = ss_edited.drop(columns=['Unnamed: 0',])
ss_edited['mp3File'] = ss_edited.audioLink.apply(lambda x: new_audio_link(x))
ss_edited = ss_edited.drop(columns=['date'])
ss_edited = ss_edited.rename(columns={'date_str':'date', 'audioLink':'oldAudioLink', 'handoutLink':'oldHandoutLink'})
ss_edited = ss_edited.reset_index().rename(columns={'index':'_id', 'series':'sundaySchoolSeries'})
ss_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(ss_edited.shape[0])]
ss_edited['_type'] = 'sundayschool'
ss_edited = ss_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sundaySchoolSeries', 'mp3File', 'oldAudioLink', 'oldHandoutLink']]
ss_edited = ss_edited.fillna("")

In [10]:
ss_edited.head(1)

,_id,_type,date,speaker,book,passage,title,sundaySchoolSeries,mp3File,oldAudioLink,oldHandoutLink
0,b653beaa-99c2-11ed-a646-000000006ec3,sundayschool,2005-09-04,Michael Gaydosh,Daniel,1:1-21,Daniel 1,Daniel,media___Daniel___Daniel_1.mp3,https://www.gracecovenantbaptist.org/media/Dan...,


In [11]:
ss_edited.groupby('speaker').count().reset_index()['speaker']

0                 All
1        Darrell Cook
2      David Wickiser
3       Dustin Curtis
4           Ed Wallen
5     Jayson Faulkner
6         Louie Davis
7         Matt Carver
8          Matt Posey
9     Michael Gaydosh
10     Richard Hughes
11       Stephen Hyde
12        Steve Cowan
13        Todd Wilson
14         Wade Potts
Name: speaker, dtype: object

In [12]:
ss_edited[ss_edited.speaker=='Darrell Cook']

,_id,_type,date,speaker,book,passage,title,sundaySchoolSeries,mp3File,oldAudioLink,oldHandoutLink
79,b653c850-99c2-11ed-8ba8-0000000182b0,sundayschool,2007-04-22,Darrell Cook,Exodus,13:1-22,Sanctified to the Lord,Exodus,media___Exodus___Exodus_13.mp3,https://www.gracecovenantbaptist.org/media/Exo...,
201,b653d688-99c2-11ed-aa8c-000000013454,sundayschool,2009-09-27,Darrell Cook,Deuteronomy,2:16-3:29,Review of God’s Dealings with Israel (Pt. 2),Deuteronomy,media___Deuteronomy___Deuteronomy_3.mp3,https://www.gracecovenantbaptist.org/media/Deu...,
209,b653d76e-99c2-11ed-87d2-000000001532,sundayschool,2009-11-22,Darrell Cook,Deuteronomy,10:1-22,Exposition of the 1st Commandment: What Does t...,Deuteronomy,media___Deuteronomy___Deuteronomy_11.mp3,https://www.gracecovenantbaptist.org/media/Deu...,
213,b653d7e6-99c2-11ed-8841-000000009312,sundayschool,2009-12-20,Darrell Cook,Deuteronomy,14:22-16:17,Exposition of the 4th Commandment: The Tribute...,Deuteronomy,media___Deuteronomy___Deuteronomy_15.mp3,https://www.gracecovenantbaptist.org/media/Deu...,
214,b653d804-99c2-11ed-9b53-00000000a49b,sundayschool,2009-12-27,Darrell Cook,Deuteronomy,16:18-18:22,Exposition of the 5th Commandment: The Leader ...,Deuteronomy,media___Deuteronomy___Deuteronomy_16.mp3,https://www.gracecovenantbaptist.org/media/Deu...,


In [ ]:
# ss_edited = ss_edited[ss_edited.date <= "2022-10-31"]

ss_edited.drop(columns=['mp3File']).to_json("../cms/sundayschool.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)